In [ ]:
!pip install git+https://www.github.com/mouseland/cellpose.git

In [ ]:
import numpy as np
from cellpose import models, core, io, plot
from pathlib import Path
from tqdm import trange
import matplotlib.pyplot as plt

io.logger_setup() # run this to get printing of progress

#Check if colab notebook instance has GPU access
if core.use_gpu()==False:
  raise ImportError("No GPU access, change your runtime")

model = models.CellposeModel(gpu=True)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
from cellpose import models, transforms, utils

# Load Cellpose model (adjust model_type if needed)
model = models.CellposeModel(gpu=True, model_type='cyto')  # ✅ New

# Path to your folder of .tif images
image_folder = 'images'  
image_files = sorted([f for f in os.listdir(image_folder) if f.endswith('.tif')])[:2]
print(image_files)

# Load a few images (e.g. 6 max for nice visualization)
imgs = []
image_names = []
for fname in sorted(image_files):
    img = imread(os.path.join(image_folder, fname))
    if img.ndim == 2:
        img = np.expand_dims(img, axis=0)  # grayscale: (1, H, W)
    elif img.ndim == 3 and img.shape[-1] == 3:
        img = img.transpose(2, 0, 1)        # RGB: (3, H, W)
    imgs.append(img)
    image_names.append(fname)

imgs = np.stack(imgs, axis=0)  # shape: (N, C, H, W)
print(imgs.shape)


In [ ]:
# Prepare output lists
masks_pred_all, flows_all, styles_all = [], [], []


for img in imgs:
  # Inference using more iterations (e.g., for bacterial images)
  masks_pred, flows, styles = model.eval(
      img,
      niter=1000,
      do_3D=False,      # ✅ force 2D
      channels=[0, 0]   # ✅ grayscale; adjust to [0,1] if using 2-channel input
  )
  masks_pred_all.append(masks_pred)
  flows_all.append(flows)
  styles_all.append(styles)

In [ ]:
# Titles for visualization
titles = image_names


# Plotting predictions
plt.figure(figsize=(14, 6))
for iex in range(len(imgs)):
    img = imgs[iex].copy()
    img = np.clip(transforms.normalize_img(img, axis=0), 0, 1)

    ax = plt.subplot(2, 3, iex + 1)
    if img.shape[0] == 1:
        ax.imshow(img[0], cmap="gray")
    else:
        ax.imshow(img.transpose(1, 2, 0))

    # Draw predicted mask outlines in yellow
    outlines_pred = utils.outlines_list(masks_pred_all[iex])
    for o in outlines_pred:
        plt.plot(o[:, 0], o[:, 1], color=[1, 1, 0.3], lw=0.75, ls="--")

    ax.set_title(titles[iex])
    ax.axis('off')


plt.tight_layout()
plt.show()
